In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, DataFrame
from pyspark.sql.types import *
from pyspark.sql.functions import udf, col, regexp_replace, concat
import  pyspark.sql.functions as F
import spacy
import stanza

SparkContext.setSystemProperty("spark.sql.crossJoin.enabled", "true")
sc = SparkContext.getOrCreate()
spark = SQLContext(sc)

merged_se_cleaned = 'merged_cleaned_files_se_before_extraction-table' 

In [ ]:
# sc.addPyFile("gs://workbench.gleanlabs.net/emr_data/user=johanna/nlp_pipeline_new_remap.zip")
sc.addPyFile("gs://workbench.gleanlabs.net/emr_data/user=johanna/nlppipeline_stanza.zip")

In [ ]:
times_nltk = []

In [ ]:
from lib.sparkmethods import stage1
import time

for N in [100,1000,5000,10000,50000]:
    start = time.time()
    df = spark.read.format('delta').load("gs://workbench.gleanlabs.net/emr_data/user=johanna/{}".format(merged_se_cleaned)).limit(N)
    df = df.repartition(1800, ['docID'])
    dfKeyPhrases = stage1(df)
    dfKeyPhrases.write.format('delta').save("gs://workbench.gleanlabs.net/emr_data/user=johanna/{}".format('merged_se_extracted_kps_new_posts_{}_stanza-table'.format(N)))
    end = time.time() - start
    times_nltk.append(end)

In [ ]:
times_nltk

In [ ]:
import pandas as pd
nltk = {'sentences': times_nltk}

df_nltk = pd.DataFrame(nltk, index = [100,1000,5000,10000,50000])

In [ ]:
df_nltk

In [ ]:
df_nltk = pd.to_csv('stanza_time.csv')